In [16]:
import json
import numpy as np
from bokeh.io import push_notebook, show, output_notebook
from bokeh.layouts import row
from bokeh.plotting import figure
from bokeh.palettes import BuPu, Greens, Set3, Blues, Oranges, viridis

In [68]:
output_notebook()
opts = dict(plot_width=1000, plot_height=400, min_border=0)

Loading BokehJS ...

In [6]:
#Load json file
#Returns list -> called an object 
def loadJson(_fileName):
    with open(_fileName) as json_data:
        jsonDump = json.load(json_data)
    json_data.close()
    
    return jsonDump

In [7]:
#Pass it an object
#Returns object
def combineFiles(_obj):
    
    numOfObjsToMerge = len(_obj)
    
    mergedObj = []
    for i in range(numOfObjsToMerge):
        
        for j in range(len(_obj[i])):
            mergedObj.append(_obj[i][j])
    
    return mergedObj

In [8]:
#Remove list item
#Returns object
def removeItem(_obj, _listNumber):
    _obj = _obj.pop(_listNumber)
    return _obj

In [61]:
#Pass it an object a list describing the data
# Example [3, 4] -> First 3 are one type (woof) and last 4 are a different type (meow)
# _formatOn must be (t/f).(Data/Freq)
#Retunrs plot figure
def plotObj(_obj, _listDiffTypes, _formatOn, opts=None):

    #For plotting
    _listDiffTypes.insert(0, 0)
    colorP = viridis(len(_listDiffTypes))

    p1 = figure(**opts)
    
    tOrF = _formatOn.split('.')[0]
    _formatOn = _formatOn.split('.')[1]

    colorNum = 0
    for i in range(len(_obj)):
        
        #Update color
        if i in _listDiffTypes:
            if i != 0:
                colorNum = colorNum + 1
        
        fft = _obj[i]['aClip'][tOrF]['fFreq']  #
        freq = _obj[i]['aClip'][tOrF]['fData'] #amp
        
        fftPlot = freq + [freq[-1], 0]
        freqPlot = fft + [len(fft)+1, len(fft)+2]
        
        freqPlot[0] = 0.0
        
        legendName = 'Data' + str(i)
        
        p1.patches( [ freqPlot ],
                    [ fftPlot ],
                    line_width=1, color=colorP[colorNum], alpha=0.1, legend=legendName)
        p1.legend.location = 'top_right'
        p1.legend.click_policy = 'hide'
    
    return p1

In [72]:
#Pass it an object
#Must combine inputs and output
#Format on can be any json key (fft, time series, etc)
#Saves to file
def formatForTraining(_obj, _fileName):
    
    toExport = []
    
    for i in range(len(_obj)):
        data = _obj[i]['aClip']['f']['fData'] 
        result = _obj[i]['aClip']['info']['output']
        
        #print data + result
        toExport = data + result
        
        f=open(_fileName,'ab')
        np.savetxt(f, [toExport], delimiter=' ', fmt='%4.4f')
        f.close()

In [78]:
#Define 
#fileName = 'woof1.json'

#Load woofs
woofs = loadJson(fileName)

#Load meows
fileName2 = 'meow1.json'
meows = loadJson(fileName2)

#Combine files
woofsAndMeows = combineFiles([meows])

#print json.dumps(woofsAndMeows)

#Remove item
#removeItem(woofsAndMeows, len(woofsAndMeows)-1 )

#Plot data
p1 = plotObj(woofsAndMeows, [len(meows)], 'f.fData', opts=opts)
show(p1)

In [79]:
#Export 
formatForTraining(woofsAndMeows, 'meow1.data')